# Automatic evaluation part 1

This is demo code for Automatic evaluation part 1.

Input files include text, image, csv table and the MC questions. The output are the chatGPT response txt file

## Package installation

In [2]:
%pip install python-docx
%pip install anthropic
# !pip install mistralai
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 18.4 MB/s eta 0:00:00


In [4]:
import os
import fnmatch
import pandas as pd
from google.colab import drive

from openai import OpenAI
#DAMN https://ai.google.dev/gemini-api/docs/migrate
#import google.generativeai as genai
from google import genai
from google.genai import types

import anthropic
# from mistralai import Mistral

import csv
import json
import docx
import openpyxl
import base64
import chardet
from typing import Union, List, Dict
import tiktoken
import time
import httpx
import numpy as np
import random

In [8]:
from google import genai
print(genai.__version__)

1.21.1


## File Reading Functions

In [ ]:
def file_to_text(file_path: str) -> str:
    """
    Converts a DOCX, XLSX, CSV, JSON, RMD (R Markdown), or PY file
    into a text string suitable for LLM input.

    :param file_path: The path to the file to be converted.
    :return: A single string containing the file’s textual contents.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    if ext == ".docx":
        return _docx_to_text(file_path)
    elif ext == ".xlsx":
        return _xlsx_to_text(file_path)
    elif ext == ".csv":
        return _csv_to_text(file_path)
    elif ext == ".json":
        return _json_to_text(file_path)
    elif ext == ".rmd":
        return _rmarkdown_to_text(file_path)
    elif ext == ".md":
        return _rmarkdown_to_text(file_path)
    elif ext == ".py":
        return _python_to_text(file_path)
    elif ext == ".txt":
        return _txt_to_text(file_path)
    else:
        raise ValueError(f"Unsupported file extension: {ext}")


def _docx_to_text(file_path: str) -> str:
    """
    Reads a DOCX file and extracts all text paragraphs.
    Requires 'python-docx' (pip install python-docx).
    """
    if docx is None:
        raise ImportError("Missing dependency 'python-docx'. Install via `pip install python-docx`.")

    doc = docx.Document(file_path)
    paragraphs = [para.text for para in doc.paragraphs]
    return "\n".join(paragraphs)


def _xlsx_to_text(file_path: str) -> str:
    """
    Reads an XLSX file and concatenates all cells as text.
    Requires 'openpyxl' (pip install openpyxl).
    """
    if openpyxl is None:
        raise ImportError("Missing dependency 'openpyxl'. Install via `pip install openpyxl`.")

    wb = openpyxl.load_workbook(file_path, data_only=True)
    all_text = []
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        all_text.append(f"--- Sheet: {sheet_name} ---")
        for row in sheet.iter_rows(values_only=True):
            row_text = [str(cell) if cell is not None else "" for cell in row]
            all_text.append("\t".join(row_text))

    return "\n".join(all_text)


def _csv_to_text(file_path: str) -> str:
    """
    Reads a CSV file line by line and returns its text representation.
    """
    rows = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            rows.append("\t".join(row))
    return "\n".join(rows)


def _json_to_text(file_path: str) -> str:
    """
    Reads a JSON file and returns its pretty-printed JSON string.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return json.dumps(data, indent=2, ensure_ascii=False)


def _rmarkdown_to_text(file_path: str) -> str:
    """
    R Markdown files are basically text files with embedded code.
    We’ll just read the raw text for simplicity.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

def _txt_to_text(file_path: str) -> str:
    """
    Reads a text file and returns its content as a string.
    Automatically detects encoding to avoid decoding errors.
    """
    #import chardet
    try:
        # Detect the file encoding
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            detected = chardet.detect(raw_data)
            encoding = detected['encoding']

        # Read the file with the detected encoding
        with open(file_path, 'r', encoding=encoding) as file:
            return file.read()
    except Exception as e:
        raise RuntimeError(f"Error reading file {file_path}: {e}")

def _python_to_text(file_path: str) -> str:
    """
    Python (.py) files are plain text. Just read the entire file content.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        code = f.read()
    return code

def encode_image(image_path: str) -> tuple[str, str]:
    """
    Encode an image file to base64 and return its MIME type.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # Read and encode image
    with open(image_path, "rb") as image_file:
        base64_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Get MIME type based on file extension
    _, extension = os.path.splitext(image_path)
    extension = extension.lower()

    mime_type = {
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.png': 'image/png',
        '.gif': 'image/gif',
        '.webp': 'image/webp'
    }.get(extension)

    if not mime_type:
        raise ValueError(f"Unsupported image format: {extension}")

    return base64_data, mime_type


## Model functions

In [ ]:
def generate_MC_answer_openai(api_key: str, dir: str, rfile_path: str, images_folder_path: str, dfile_path: str, system_prompt: str = None, model: str = "gpt-4o") -> str:
    """
    Generates a result paragraph based on the given report and multiple-choice document using OpenAI's API.

    Args:
        api_key (str): Your OpenAI API key.
        report_txt (str): The main report content to analyze.
        mc_doc (str): The multiple-choice document content for the question.
        system_prompt (str, optional): The system-level instruction for the AI. Defaults to a biological writing prompt.
        model (str, optional): The OpenAI model to use. Defaults to "gpt-4o".

    Returns:
        str: The generated result paragraph.
    """


    # Initialize OpenAI client
    client = OpenAI(api_key=api_key)

    # Use a default system prompt if none is provided
    if system_prompt is None:
        system_prompt = (
            "I am a bioinformatician that has conducted some analyses but lacks the biological knowledge to interpret the results and build a cohesive narrative."
            "However, you are an expert scientist who has a strong background in both bioinformatics and biology."
            "I would like to ask you to interpret and translate my bioinformatics analyses into a coherent 'Results' section suitable for a high-impact interdisciplinary scientific journal."
					  "I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots."
            "These files are the core analytics you will need to synthesize and summarize into a coherent narrative. The .txt file also includes an 'LLM' detailing any specific requirements or instructions for your summary. Please read it carefully."
            "Please remember the following when completing this request: Interpret and synthesize the provided bioinformatics results into a clear and logical narrative, complete any specific requests outlined in the LLM task, include quantitative evidence from the analyses to support your conclusions, and include relevant literature to support or contextualise your findings in your summary."
        )

    # Prepare the input message
    user_prompt=[]

    user_prompt.append(
       "I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots. Please synthesize and summarize the contents of the file(s) into a coherent narrative. Be sure to read the 'LLM task' in the .txt file.\n"
        )

    if pd.notna(rfile_path):
        report_txt = file_to_text(os.path.join(dir,rfile_path))
        user_prompt.append("Txt file:\n" + report_txt)

    # if pd.notna(dfile_path):
    #     data_txt = file_to_text(os.path.join(dir,dfile_path))
    #     user_prompt.append("I have a csv file:" + "\n" + data_txt)

    if pd.notna(dfile_path):
        # Split dfile_path into individual file paths
        file_paths = dfile_path.split("\n")
        if len(file_paths)>1:
            user_prompt.append("CSV files:\n")
        else:
            user_prompt.append("CSV file:\n")
        for file_path in file_paths:
            # Process each file path
            full_path = os.path.join(dir, file_path.strip())  # Strip any extra spaces or newline characters
            if os.path.exists(full_path):  # Ensure the file exists before reading
                data_txt = file_to_text(full_path)  # Convert the file to text
                user_prompt.append(data_txt+"\n")
            else:
                print(f"File not found: {full_path}")


    if pd.notna(images_folder_path):
        image_path_list = []
        image_dir=os.path.join(dir,images_folder_path,"figure-markdown_strict")
        for file_name in os.listdir(image_dir):
            if file_name.endswith((".jpg", ".jpeg", ".png", ".gif", ".webp")):
                image_path_list.append(os.path.join(image_dir, file_name))
        image_file_list = [encode_image(item) for item in image_path_list]
        for image_file,mime_type in image_file_list:
            user_prompt.append("Image file:\n")
            user_prompt.append(
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:{mime_type};base64,{image_file}",
                    }
                }
            )



    messages = [
        {"role": "system", "content": f"{system_prompt}"},
        {"role": "user", "content": f"{user_prompt}"},
    ]

    print("Number of tokens(Lijia):", count_tokens(messages,model=model))
    print("Number of tokens(Xumou):", num_tokens_from_messages(messages,model=model))


    #Generate the completion
    try:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=model,
            # temperature=0.1
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        # raise RuntimeError(f"Error generating result paragraph: {e}")
        print(f"Exception occurred: {e}")
            # Return a default error string
        return "error"



In [ ]:
def generate_MC_answer_google_gemini(api_key: str, dir: str, rfile_path: str, images_folder_path: str, dfile_path: str, system_prompt: str = None, model: str = "gemini-2.0-flash-exp") -> str:
    """
    Generates a result paragraph based on the given report and multiple-choice document using Google's Generative AI API.

    Args:
        api_key (str): Your Google Generative AI API key.
        report_txt (str): The main report content to analyze.
        mc_doc (str): The multiple-choice document content for the question.
        system_prompt (str, optional): The system-level instruction for the AI. Defaults to a biological writing prompt.
        model (str, optional): The Google Generative AI model to use. Defaults to "models/chat-bison-001".

    Returns:
        str: The generated result paragraph.
    """

    # Configure the API key
    client = genai.Client(api_key=api_key)

    # Use a default system prompt if none is provided
    if system_prompt is None:
        system_prompt = (
            "I am a bioinformatician that has conducted some analyses but lacks the biological knowledge to interpret the results and build a cohesive narrative."
            "However, you are an expert scientist who has a strong background in both bioinformatics and biology."
            "I would like to ask you to interpret and translate my bioinformatics analyses into a coherent 'Results' section suitable for a high-impact interdisciplinary scientific journal."
					  "I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots."
            "These files are the core analytics you will need to synthesize and summarize into a coherent narrative. The .txt file also includes an 'LLM' detailing any specific requirements or instructions for your summary. Please read it carefully."
            "Please remember the following when completing this request: Interpret and synthesize the provided bioinformatics results into a clear and logical narrative, complete any specific requests outlined in the LLM task, include quantitative evidence from the analyses to support your conclusions, and include relevant literature to support or contextualise your findings in your summary."
        )

    # Prepare the input message
    user_prompt=[]
    user_prompt.append("I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots. Please synthesize and summarize the contents of the file(s) into a coherent narrative. Be sure to read the 'LLM task' in the .txt file. \n")


    if pd.notna(rfile_path):
        report_txt = file_to_text(os.path.join(dir,rfile_path))
        user_prompt.append("Txt file:\n" + report_txt)

    # if pd.notna(dfile_path):
    #     data_txt = file_to_text(os.path.join(dir,dfile_path))
    #     user_prompt.append("I have a csv file:" + "\n" + data_txt)

    if pd.notna(dfile_path):
        # Split dfile_path into individual file paths
        file_paths = dfile_path.split("\n")
        if len(file_paths)>1:
            user_prompt.append("CSV files:\n")
        else:
            user_prompt.append("CSV file:\n")
        for file_path in file_paths:
            # Process each file path
            full_path = os.path.join(dir, file_path.strip())  # Strip any extra spaces or newline characters
            if os.path.exists(full_path):  # Ensure the file exists before reading
                data_txt = file_to_text(full_path)  # Convert the file to text
                user_prompt.append(data_txt+"\n")
            else:
                print(f"File not found: {full_path}")


    if pd.notna(images_folder_path):
        image_path_list = []
        image_dir=os.path.join(dir,images_folder_path,"figure-markdown_strict")
        for file_name in os.listdir(image_dir):
            if file_name.endswith((".jpg", ".jpeg", ".png", ".gif", ".webp")):
                image_path_list.append(os.path.join(image_dir, file_name))
        image_file_list = [encode_image(item) for item in image_path_list]
        for image_file,mime_type in image_file_list:
            user_prompt.append("Image file:\n")
            user_prompt.append(
                # {'mime_type': mime_type, 'data': image_file}
                # image_file
                types.Part.from_bytes(data=image_file, mime_type=mime_type)
            )

    # user_prompt.append("Please summarise all the information above and answer the following question:" + "\n"
    #                    + mc_txt +
    #                    "\nPlease provide only the letter of the correct option (A, B, C, D, or E). \
    #                    Do not include the answer text, explanations, or any other information.")

    # messages = [
    #     {"role": "user", "parts": f"{user_prompt}"},
    # ]
    # print(user_prompt)

        # Create the model
    # generation_config = {
    #   # "temperature": 0.1,
    #   "top_p": 0.95,
    #   "top_k": 40,
    #   "max_output_tokens": 8192,
    #   "response_mime_type": "text/plain",
    # }

    try:
        # chat_session = model.start_chat()
        # chat_session = model.start_chat(history=messages)
        # Send an additional message if needed (optional)
        # response = chat_session.send_message(messages)
        response = client.models.generate_content(
            model=model,
            contents=user_prompt,
            config=types.GenerateContentConfig(
                # generation_config=generation_config,
                # temperature= 0.5,
                response_mime_type="text/plain",
                system_instruction=system_prompt,
            )
        )

        return response.text
    except Exception as e:
        # raise RuntimeError(f"Error generating result paragraph with Generative AI: {e}")
        print(f"Exception occurred: {e}")
            # Return a default error string
        return "error"



In [ ]:
def generate_MC_answer_claude(api_key: str, dir: str, rfile_path: str, images_folder_path: str, dfile_path: str,  system_prompt: str = None, model: str = "claude-3-5-sonnet-20241022") -> str:
    """
    Generates a multiple-choice answer based on the given report and document using Claude's API.

    Args:
        api_key (str): Your Claude API key.
        report_txt (str): The main report content to analyze.
        mc_doc (str): The multiple-choice document content for the question.
        system_prompt (str, optional): The system-level instruction for the AI. Defaults to a biological writing prompt.
        model (str, optional): The Claude model to use. Defaults to "claude-3-5-sonnet-20241022".

    Returns:
        str: The multiple-choice answer generated by Claude.
    """
    # Initialize the Claude API client
    client = anthropic.Anthropic(
        api_key=api_key
    )

    # Use a default system prompt if none is provided
    if system_prompt is None:
        system_prompt = (
            "I am a bioinformatician that has conducted some analyses but lacks the biological knowledge to interpret the results and build a cohesive narrative."
            "However, you are an expert scientist who has a strong background in both bioinformatics and biology."
            "I would like to ask you to interpret and translate my bioinformatics analyses into a coherent 'Results' section suitable for a high-impact interdisciplinary scientific journal."
					  "I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots."
            "These files are the core analytics you will need to synthesize and summarize into a coherent narrative. The .txt file also includes an 'LLM' detailing any specific requirements or instructions for your summary. Please read it carefully."
            "Please remember the following when completing this request: Interpret and synthesize the provided bioinformatics results into a clear and logical narrative, complete any specific requests outlined in the LLM task, include quantitative evidence from the analyses to support your conclusions, and include relevant literature to support or contextualise your findings in your summary."
        )

    # Prepare the input message
    user_prompt=[]

    user_prompt.append({"type": "text", "text":"I have provided you with a .txt file containing the outputs from my analysis, and in some cases, additional supporting materials such as plots. Please synthesize and summarize the contents of the file(s) into a coherent narrative. Be sure to read the 'LLM task' in the .txt file.\n"})


    if pd.notna(rfile_path):
        report_txt = file_to_text(os.path.join(dir,rfile_path))
        user_prompt.append({"type": "text", "text":"Txt file:\n" + report_txt})

    # if pd.notna(dfile_path):
    #     data_txt = file_to_text(os.path.join(dir,dfile_path))
    #     user_prompt.append("I have a csv file:" + "\n" + data_txt)

    if pd.notna(dfile_path):
        # Split dfile_path into individual file paths
        file_paths = dfile_path.split("\n")
        if len(file_paths)>1:
            user_prompt.append({"type": "text", "text":"CSV files:\n"})
        else:
            user_prompt.append({"type": "text", "text":"CSV file:\n"})
        for file_path in file_paths:
            # Process each file path
            full_path = os.path.join(dir, file_path.strip())  # Strip any extra spaces or newline characters
            if os.path.exists(full_path):  # Ensure the file exists before reading
                data_txt = file_to_text(full_path)  # Convert the file to text
                user_prompt.append({"type": "text", "text": data_txt+"\n"})
            else:
                print(f"File not found: {full_path}")


    if pd.notna(images_folder_path):
        image_path_list = []
        image_dir=os.path.join(dir,images_folder_path,"figure-markdown_strict")
        for file_name in os.listdir(image_dir):
            if file_name.endswith((".jpg", ".jpeg", ".png", ".gif", ".webp")):
                image_path_list.append(os.path.join(image_dir, file_name))
        image_file_list = [encode_image(item) for item in image_path_list]
        for image_file,mime_type in image_file_list:
            user_prompt.append( "Image file:\n")
            user_prompt.append(
                {
                    "type": "image",
                    "source": {
                    "type": "base64",
                    "media_type": mime_type,  # e.g., "image/jpeg"
                    "data": image_file
                    }
                }
            )


    messages = [
        {"role": "user", "content": f"{user_prompt}"},
    ]

    # print(messages)

    # Call the Claude API to generate the response
    try:
        response = client.messages.create(
            model=model,
            max_tokens=8192,
           # temperature=0.1,
            system=system_prompt,
            messages=messages
        )
        return response.content[0].text # Extract the generated content
    except Exception as e:
        # raise RuntimeError(f"Error generating multiple-choice answer with Claude: {e}")
        print(f"Exception occurred: {e}")
            # Return a default error string
        return "error"



## Mount google drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Read Prompt Sheet

In [ ]:
## Don't change this url
url = 'https://docs.google.com/spreadsheets/d/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/export?format=csv&gid=XXXXXXXXXXXXXXX'
case_df = pd.read_csv(url)
print(case_df.head(6))
df=case_df

     Authors          Google Folder              CaseStudy_ID  \
0  EXAMPLE 1  Bioinformatics_method  Dataset_method_increment   
1  EXAMPLE 2  Bioinformatics_method  Dataset_method_increment   
2  EXAMPLE 3  Bioinformatics_method  Dataset_method_increment   
3         DK                Pathway          Kidney_pathway_1   
4         DK                     DE               Kidney_DE_1   
5         DK                Pathway          Kidney_pathway_2   

                   Sample_ID LLM task updated?    Input type  \
0  Dataset_method_incrementA                No         Graph   
1   Dataset_method_increment                No          Data   
2  Dataset_method_incrementB                No   Code + Data   
3          Kidney_pathway_1B                No  Code + Graph   
4                Kidney_DE_1                No   Code + Data   
5           Kidney_pathway_2                No   Code + Data   

       RMD_Code_input_ID           TXT_input_ID   Graphics_input_Folder  \
0                   

### Parameters

In [ ]:
OPENAI_api_key= "sk-"
GEMINI_api_key= "AI"
CLAUDE_api_key="sk-"
## root_dir is the only path that you need to modify by yourself.
## You may find the shared Proj-LLM-Bioinfo-Interpretation2024 folder in the /content/drive/MyDrive,
## so the dir path can be /content/drive/MyDrive/Proj-LLM-Bioinfo-Interpretation2024/Rmd_word_document/
root_dir = "/content/drive/MyDrive/Usyd/Proj-LLM-Bioinfo-Interpretation2024/Rmd_word_document/"

In [ ]:
#### test
os.path.join(root_dir, df['Google Folder'][55], df['CaseStudy_ID'][55],df['TXT_input_ID'][55])

'/content/drive/MyDrive/Usyd/Proj-LLM-Bioinfo-Interpretation2024/Rmd_word_document/CCI/Covid_CCI_8/CCI_chua_ligand_receptor_contribution_CXCL_only_plot.txt'

### Multiple Choice questions


In [ ]:
columns_to_check = ['TXT_input_ID', 'Graphics_input_Folder', 'Data_input_ID']
index=np.where(case_df["Authors"] == "DK")[0]
index

array([  3,   4,   5,  10,  57,  58,  59,  79,  80, 100, 101, 102, 103,
       112, 113, 114, 115, 116, 117, 118, 119])

In [ ]:
columns_to_check = ['TXT_input_ID', 'Graphics_input_Folder', 'Data_input_ID']
index=np.where(case_df["Authors"] == "JG")[0]
for idx in index: #range(len(mc_df)):range(18,19), range(len(case_df))[5:]
    if pd.notna(case_df['Google Folder'][idx]) and pd.notna(case_df['Sample_ID'][idx]):
        subfolder = os.path.join(root_dir, case_df['Google Folder'][idx], case_df['CaseStudy_ID'][idx])
        row_entry=case_df[case_df['Sample_ID'] == case_df['Sample_ID'][idx]].iloc[0]
        # print(row_entry)
        print(idx)
        print(row_entry["Sample_ID"])
        gpt4o_answer=generate_MC_answer_openai(api_key=OPENAI_api_key, dir=subfolder,
                                  rfile_path=row_entry["TXT_input_ID"], images_folder_path=row_entry["Graphics_input_Folder"],
                                  dfile_path=row_entry["Data_input_ID"],  model = "gpt-4o")
        gemini_answer=generate_MC_answer_google_gemini(api_key=GEMINI_api_key, dir=subfolder,
                                  rfile_path=row_entry["TXT_input_ID"], images_folder_path=row_entry["Graphics_input_Folder"],
                                  dfile_path=row_entry["Data_input_ID"],  model = "gemini-2.0-flash")
        claude_answer=generate_MC_answer_claude(api_key=CLAUDE_api_key, dir=subfolder,
                                  rfile_path=row_entry["TXT_input_ID"], images_folder_path=row_entry["Graphics_input_Folder"],
                                  dfile_path=row_entry["Data_input_ID"],  model = "claude-3-7-sonnet-20250219")

        time.sleep(60)
        model_answers = {
                        "gpt-4o": gpt4o_answer,
                        "gemini-2.0-flash": gemini_answer,
                        "claude-3-7-sonnet-20250219": claude_answer
                    }
        for j in model_answers.keys():
            file_path="/content/drive/MyDrive/Usyd/Proj-LLM-Bioinfo-Interpretation2024/Report_output/"+row_entry['Sample_ID']+"."+j+".txt"
            # print(file_path)
            with open(file_path, 'w') as file:
                file.write(model_answers[j])
            print(f"File saved to: {file_path}")

